In [ ]:
import pyspark
from pyspark.sql import SparkSession

# Create a Spark session (Control center of Spark functionality)
spark = SparkSession.builder.appName("MySparkApp").getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", "5")

# A Spark Overview


- Running apps with **spark-submit**
- Datasets: type-safe APIs
- Structured Streaming
- Machine learning and advanced analytics
- RDD: Spark low level APIs
- SparkR
- Third-party package ecosystem


## spark-submit

A build-in command-line that sends your application code to a cluster and lunch it execute it there.

The application will run there until it completes it's task and gets an error

**spark-submit main.py**

| Aspect              | `spark-submit main.py` | `python main.py`      |
| ------------------- | ---------------------- | --------------------- |
| Starts Spark JVM    | ✅ Yes                 | ❌ Indirect / fragile |
| Cluster execution   | ✅ Yes                 | ❌ No                 |
| Config control      | ✅ Full                | ❌ Limited            |
| Executor management | ✅ Yes                 | ❌ No                 |
| Dependency shipping | ✅ Yes                 | ❌ No                 |
| Production-ready    | ✅ Yes                 | ❌ No                 |
| Notebooks           | ❌                     | ✅                    |


## Datasets

Only available in Java and Scala


## Structured Streaming

A stream processing engine built on top of Spark SQL. It allows you to process real-time data streams using the same DataFrame/Dataset API that Spark uses for batch processing.

Streams are treated as tables that grow continuously.
Each record in the stream is like a new row in a continuously appending table.

Structured Streaming processes data in micro-batches or in continuous mode:

- Micro-batch mode (default): Spark processes incoming data in small batches at intervals.
- Continuous mode: Spark processes data as it arrives (very low latency, experimental).


In [ ]:
# loads all the csv files from the path
# we can specify just the folder or use wildcards  to extract just certain files
staticDataFrame = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("../data/retail-data/by-day")
staticDataFrame.createOrReplaceTempView("retail_data")
staticSchema = staticDataFrame.schema

#either form works
staticDF2011 = spark.read.csv("../data/retail-data/by-day/2011*.csv", header=True, inferSchema=True)
staticDF2010 = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("../data/retail-data/by-day/2010*.csv")
staticDataFrame.createOrReplaceTempView("retail_data")


In [ ]:
from pyspark.sql.functions import window, column, desc, col
staticDataFrame\
.selectExpr(
"CustomerId",
"(UnitPrice * Quantity) as total_cost",
"InvoiceDate")\
.groupBy(
col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")\
.show(5)


staticDataFrame.createOrReplaceTempView("retail_data")
spark.sql("""
SELECT CustomerId,
       window(InvoiceDate, '1 day') as day_window,
       SUM(UnitPrice * Quantity) as total_cost
FROM retail_data
GROUP BY CustomerId, window(InvoiceDate, '1 day')
""").show(5)

staticDF2011.createOrReplaceTempView("retail_data")
spark.sql("""
SELECT COUNT(*) as Data_2011
FROM retail_data
""").show(1)

staticDF2010.createOrReplaceTempView("retail_data")
spark.sql("""
SELECT COUNT(*) as Data_2010
FROM retail_data
""").show(1)

spark.read.csv:

- will only read the files at the path once
- can infer schema (inferSchema=True) or you can specify it
- returns a DataFrame that is finite (all data is already there)

spark.readStream:

- will reads new files that appear in a folder continuously
- schema must be explicitly defined (inferSchema is not supported in streaming)
- returns a DataFrame representing an unbounded table


In [ ]:
staticSchema = staticDataFrame.schema

# maxFilesPerTrigger number of files we will read at once (In PRD scenarios usually omitted)
streamingDataFrame = spark.readStream\
.schema(staticSchema)\
.option("maxFilesPerTrigger", 1)\
.format("csv")\
.option("header", "true")\
.load("../data/retail-data/by-day/*.csv")

In [ ]:
streamingDataFrame.isStreaming # returns True

In [ ]:
purchaseByCustomerPerHour = streamingDataFrame\
.selectExpr(
"CustomerId",
"(UnitPrice * Quantity) as total_cost",
"InvoiceDate")\
.groupBy(
col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")

In [ ]:
# creates table customer_purchases in memory and constantly updates it with ne files added
purchaseByCustomerPerHour.writeStream\
.format("memory")\
.queryName("customer_purchases")\
.outputMode("complete")\
.start()

In [ ]:
# current running stream
for q in spark.streams.active:
    print(q.name, q.id, q.status)


In [ ]:
# we query the table customer_purchases
spark.sql("""
SELECT *
FROM customer_purchases
ORDER BY `sum(total_cost)` DESC
""")\
.show(5)

spark.sql("""
SELECT COUNT(*) AS num_rows
FROM customer_purchases
""")\
.show(5)

In [ ]:
for q in spark.streams.active:
    q.stop()

## Machine Learning and Advanced Analytics


Uses build in library of machine learning **MLlib**

You can even use models trained in MLlib to make predictions in Structured Streaming


MLlib require that the data is represented in numerical values

Currently our data is represented by a variety of types


In [ ]:
staticDataFrame.printSchema()

In [ ]:
from pyspark.sql.functions import date_format, col
# na.fill() - replace NULL with 0

preppedDataFrame = staticDataFrame\
.na.fill(0)\
.withColumn("day_of_week", date_format(col("InvoiceDate"), "EEEE"))\
.coalesce(5)

We split data into training sets

We will split them manually by date of purchase

We could also use MLlib’s transformation API to create training and test set


In [ ]:
trainDataFrame = preppedDataFrame\
.where("InvoiceDate < '2011-07-01'")
testDataFrame = preppedDataFrame\
.where("InvoiceDate >= '2011-07-01'")

In [ ]:
trainDataFrame.count()


In [ ]:
testDataFrame.count()

Spark’s MLlib also provides a number of transformations with which we can automate
some of our general transformations. One such transformer is a StringIndexer:


In [ ]:
# turn days of week into corresponding numeric value
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()\
.setInputCol("day_of_week")\
.setOutputCol("day_of_week_index")

# encodes the numeric values (If Saturday=6 and Monday=1 we do not want Saturday > Monday)
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder()\
.setInputCol("day_of_week_index")\
.setOutputCol("day_of_week_encoded")

Each of these will result in a set of columns that we will “assemble” into a vector

All machine learning algorithms in Spark take as input a Vector type, which must be a set of numerical values:


In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler()\
.setInputCols(["UnitPrice", "Quantity", "day_of_week_encoded"])\
.setOutputCol("features")

Three key feature: "UnitPrice", "Quantity", "day_of_week_encoded"

Next we set this up into a pipeline so that any future data we need to transform can go through the exact same process


In [ ]:
# in Python
from pyspark.ml import Pipeline
transformationPipeline = Pipeline()\
.setStages([indexer, encoder, vectorAssembler])

**StringIndexer** needs to know how many unique values there are to be indexed


In [ ]:
fittedPipeline = transformationPipeline.fit(trainDataFrame)

After we fit the training data, we are ready to take that fitted pipeline and use it to transform all of our data in a consistent and repeatable way


In [ ]:
transformedTraining = fittedPipeline.transform(trainDataFrame)

In [ ]:
transformedTraining.cache()

In [ ]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans()\
.setK(20)\
.setSeed(1)


In [ ]:
kmModel = kmeans.fit(transformedTraining)

In [ ]:
transformedTest = fittedPipeline.transform(testDataFrame)

## Lower-Level APIs

Everything in Spark is built on top of RDDs


In [ ]:
from pyspark.sql import Row
spark.sparkContext.parallelize([Row(1), Row(2), Row(3)]).toDF()

In [ ]:
# Stop the Spark session - good practice to free up resources on low-RAM machines
spark.stop()